In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

In [2]:
tickers = [
    "FUTU",
    "NVDA",
    "ASTS",
    "TMF",
    "LABU",
    "GDXU",
    "TSLA",
]
len(tickers)

7

In [3]:
data = yf.download(tickers, start="2024-01-01", end="2024-11-11", group_by='ticker')

if len(tickers) == 1:
    # Create a MultiIndex for the columns
    multi_index_columns = pd.MultiIndex.from_tuples([(tickers[0], col) for col in data.columns])

    # Assign the new MultiIndex to the DataFrame
    data.columns = multi_index_columns

[*********************100%***********************]  7 of 7 completed


In [4]:
returns_list = []

# Loop through each stock ticker and calculate returns
for stock in tickers:
    # Access the 'Adj Close' prices using xs method
    adjusted_close = data[stock]['Adj Close']
    
    # Calculate percentage change
    returns_series = adjusted_close.pct_change()
    
    # Append the Series to the list
    returns_list.append(returns_series.rename(stock))  # Rename for clarity

# Concatenate all return Series into a single DataFrame
returns = pd.concat(returns_list, axis=1)

In [5]:
dates = returns.index

In [6]:
adj_close = data.xs("Adj Close", level=1, axis=1)
adj_close = adj_close.reindex(columns=returns.columns)

In [7]:
columns = pd.MultiIndex.from_product([tickers, ['Adj Close', 'Returns', "MA", "RSI", "EMA_12", "EMA_26", "MACD"]])
df = pd.DataFrame(index=dates, columns=columns)
df.columns = columns
for stock in tickers:
    df[(stock, "Adj Close")] = adj_close[stock]
    df[(stock, "Returns")] = returns[stock]
df = df.reset_index()

In [8]:
def EMA(w, price, last):
    a = 2/(1+w)
    return a*price + (1-a)*last
def MA(prices):
    return sum(prices) / 28
def MACD(long, short):
    return sum(long) - sum(short)
def RSI(returns):
    avg_gain = returns[returns > 0].mean()
    avg_loss = -returns[returns < 0].mean()
    return 100 * (1 - 1/(1+avg_gain/avg_loss))

In [9]:
for stock in tickers:
    df[(stock, "MA")] = df[(stock, "Adj Close")].rolling(window=28).apply(MA)

    df.loc[0, (stock, "EMA_12")] = df.loc[0, (stock, "Adj Close")]
    df.loc[0, (stock, "EMA_26")] = df.loc[0, (stock, "Adj Close")]
    for i in range(1, len(df)):
        df.loc[i, (stock, "EMA_12")] = EMA(12, df.loc[i, (stock, "Adj Close")], df.loc[i-1, (stock, "EMA_12")])
        df.loc[i, (stock, "EMA_26")] = EMA(26, df.loc[i, (stock, "Adj Close")], df.loc[i-1, (stock, "EMA_26")])

    df[(stock, "MACD")] = df[(stock, "EMA_26")].rolling(window=9).sum() - df[(stock, "EMA_12")].rolling(window=9).sum()
    df[(stock, "RSI")] = df[(stock, "Returns")].rolling(14).apply(RSI)

In [ ]:
# throw away days with null indicators (consider it as past data for computing the first day of our data)
close_data = df.drop(df.index[:27])
close_data = close_data.reset_index(drop=True)

# keep the 4 indicators (close price, MA, RSI, MACD)
to_drop = ["EMA_12", "EMA_26", "Returns"]
close_data = close_data.drop(columns=[(stock, label) for stock in tickers for label in to_drop])
close_data

Date        FUTU                                          NVDA  \
                 Adj Close          MA        RSI       MACD   Adj Close   
0   2024-02-09   48.529999   48.287857  77.296572   7.032787   72.117523   
1   2024-02-12   48.730000   48.189286  70.926465   6.502886   72.232498   
2   2024-02-13   47.980000   48.040357  66.847267   5.913102   72.112518   
3   2024-02-14   49.970001   47.954643  68.822373   5.138734   73.884148   
4   2024-02-15   50.689999   47.925357  64.758355   4.115298   72.642410   
..         ...         ...         ...        ...        ...         ...   
185 2024-11-04   96.900002  100.615714  52.047927 -21.390627  136.050003   
186 2024-11-05   99.080002  101.244643  49.603585 -19.759150  139.910004   
187 2024-11-06   96.654999  101.636607  56.087582 -18.935011  145.610001   
188 2024-11-07  108.660004  102.101250  64.765701 -19.356269  148.880005   
189 2024-11-08   94.510002  101.635893  47.123182 -19.276672  147.630005   

                                            ASTS  ...       LABU             \
             MA        RSI       MACD  Adj Close  ...        RSI       MACD   
0     59.275066  65.508780 -35.645876   3.430000  ...  48.323832  21.512926   
1     60.134881  65.317315 -37.043064   3.520000  ...  52.736234  19.283720   
2     61.011800  69.480824 -38.561065   3.170000  ...  45.093062  17.143178   
3     61.936673  71.333000 -40.178116   3.320000  ...  48.623972  14.740827   
4     62.777956  68.652287 -41.505414   3.410000  ...  45.424597  11.213866   
..          ...        ...        ...        ...  ...        ...        ...   
185  133.136428  43.355184 -44.198636  23.940001  ...  56.434839  12.164615   
186  133.703214  42.809788 -41.624088  24.129999  ...  58.018406  10.943712   
187  134.567857  48.216133 -39.547457  22.520000  ...  61.430927   8.330826   
188  135.547857  50.374606 -38.055313  21.455000  ...  61.720256   4.441746   
189  136.641786  49.140720 -37.019754  21.990000  ...  67.281301  -0.363792   

          GDXU                                         TSLA              \
     Adj Close         MA        RSI       MACD   Adj Close          MA   
0    20.610001  25.024286  58.956469  15.613492  193.570007  209.013572   
1    21.469999  24.647500  57.119566  15.535878  188.130005  206.860358   
2    17.920000  24.241786  51.073651  15.723208  184.020004  204.916430   
3    18.299999  23.847857  47.490553  16.306635  188.710007  203.158573   
4    19.799999  23.513571  47.910602  16.859501  200.449997  201.835716   
..         ...        ...        ...        ...         ...         ...   
185  46.549999  50.597500  54.011017 -20.870658  242.839996  241.431429   
186  47.080002  50.350357  52.984665 -17.456213  251.440002  241.332144   
187  42.099998  50.104642  50.310784 -13.506020  288.529999  242.334644   
188  45.049999  50.042857  44.487365  -9.595117  296.910004  243.594644   
189  43.169998  49.829642  49.333293  -5.689366  321.220001  245.851787   

                            
           RSI        MACD  
0    26.804465  124.197783  
1    29.850605  121.213297  
2    28.434835  117.754531  
3    42.264314  113.714704  
4    50.469826  108.253362  
..         ...         ...  
185  87.469654  -24.320575  
186  88.524769  -34.205913  
187  88.975432  -43.798632  
188  86.184263  -53.026446  
189  85.289106  -64.335344  

[190 rows x 29 columns]

In [11]:
corr = {}
for indicator in ["Adj Close", "MA", "RSI", "MACD"]:
    corr[indicator] = close_data.filter([(stock, indicator) for stock in tickers], axis=1).corr()

In [12]:
from collections import defaultdict

F = defaultdict(dict) # 4 * n * (m * n)
n = len(tickers)
for t in range(0, len(close_data) // 10): # t
    m = close_data[t*10:(t+1)*10] # m days
    for indicator in ["Adj Close", "MA", "RSI", "MACD"]: # the 4 dimensions
        for stock in tickers: # n assets
            F[t][(stock, indicator)] = m[(stock, indicator)].values.reshape(10,1).dot(corr[indicator][(stock, indicator)].values.reshape(1,n)) # m * n tensor for indicator i & stock n